In [1]:
import pandas as pd
import my_data_frequency_conversor as dfc
#import my_models as mm
#import my_llm_ts_preprocessor as tsp
import my_prompting_tool as mpt 

In [2]:
# crear un diccionario con clave dataset y valor datos de training
datasets = {
    'ETH_HOURS':'DATA/ETHUSD-H1.csv',
    'ETH_DAYS':'DATA/ETHUSD-D1.csv',
    'ETH_MONTHS':'DATA/ETHUSD-Monthly.csv',
    
}
freqs = {
    'ETH_HOURS':'h',
    'ETH_DAYS':'d',
    'ETH_MONTHS':'MS'
}
train_data = {}
val_data = {}

In [3]:
# Itera sobre cada conjunto de datos en el diccionario 'datasets'
for ds_name in datasets:  
    # Carga el archivo CSV correspondiente en un DataFrame de pandas
    df = pd.read_csv(datasets[ds_name])  
    
    # Divide los datos en entrenamiento y validación usando la función split_data de dfc
    train_data[ds_name], val_data[ds_name] = dfc.split_data(df)  
    
    # Ajusta la frecuencia de los datos de entrenamiento según el diccionario 'freqs'
    train_data[ds_name] = train_data[ds_name].asfreq(freqs[ds_name])  
    
    # Ajusta la frecuencia de los datos de validación según el diccionario 'freqs'
    val_data[ds_name] = val_data[ds_name].asfreq(freqs[ds_name])  
    
    # Redondea los valores de los datos de entrenamiento a 3 decimales
    train_data[ds_name] = train_data[ds_name].round(3)  
    
    # Redondea los valores de los datos de validación a 3 decimales
    val_data[ds_name] = val_data[ds_name].round(3)  
    
    # Imprime la forma (cantidad de filas y columnas) del conjunto de entrenamiento
    print(f'{ds_name} train data shape: {train_data[ds_name].shape}')  
    
    # Muestra las primeras filas del conjunto de entrenamiento
    print(train_data[ds_name].head())  
    
    # Imprime la forma del conjunto de validación
    print(f'{ds_name} val data shape: {val_data[ds_name].shape}')  
    
    # Muestra las primeras filas del conjunto de validación
    print(val_data[ds_name].head())  


ETH_HOURS train data shape: (26280, 1)
                           y
DateTime                    
2021-01-01 00:00:00  736.220
2021-01-01 01:00:00  733.981
2021-01-01 02:00:00  748.970
2021-01-01 03:00:00  744.460
2021-01-01 04:00:00  745.960
ETH_HOURS val data shape: (8784, 1)
                          y
DateTime                   
2024-01-01 00:00:00  2286.5
2024-01-01 01:00:00  2295.6
2024-01-01 02:00:00  2305.4
2024-01-01 03:00:00  2295.8
2024-01-01 04:00:00  2275.4
ETH_DAYS train data shape: (1095, 1)
                  y
DateTime           
2021-01-01   736.22
2021-01-02   729.46
2021-01-03   774.82
2021-01-04   976.32
2021-01-05  1041.20
ETH_DAYS val data shape: (366, 1)
                 y
DateTime          
2024-01-01  2286.5
2024-01-02  2352.7
2024-01-03  2358.2
2024-01-04  2211.3
2024-01-05  2271.0
ETH_MONTHS train data shape: (36, 1)
                   y
DateTime            
2021-01-01   736.220
2021-02-01  1313.869
2021-03-01  1424.400
2021-04-01  1919.300
2021-05-01  2769.40

In [ ]:
for dataset in datasets:
    print("-" * 50)  # Línea divisoria
    print(f'Processing {dataset} dataset')
    train, test, seq = mpt.paraphrase_nlp(dataset, train_data[dataset], val_data[dataset])
    print()




--------------------------------------------------
Processing ETH_HOURS dataset
Longitud del entrenamiento: (26280, 1)
Longitud de la prueba: (8784, 1)
Longitud de la secuencia predicha: (8781, 1)

--------------------------------------------------
Processing ETH_DAYS dataset
Longitud del entrenamiento: (1095, 1)
Longitud de la prueba: (366, 1)
Longitud de la secuencia predicha: (366, 1)

--------------------------------------------------
Processing ETH_MONTHS dataset
Longitud del entrenamiento: (36, 1)
Longitud de la prueba: (12, 1)
Longitud de la secuencia predicha: (12, 1)



: 

In [ ]:
print()

In [ ]:
# contar el número de tokens de args
import tiktoken as tk

encoding = tk.get_encoding("cl100k_base")  # Compatible con GPT-4 y GPT-3.5-turbo

# Contar tokens
num_tokens = len(encoding.encode(text))
print(f"Número de tokens: {num_tokens}")